<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/AI/Video/WAN2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt -y install -qq aria2

%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!git reset --hard 9af6320ec9d288856f505153ed03df7dfcd39dce

!pip install -r requirements.txt

# ComfyUI Managerのインストール
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git

%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/rgthree/rgthree-comfy
%cd rgthree-comfy
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes

%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF.git
%cd ComfyUI-GGUF
!pip install -r requirements.txt
%cd /content/ComfyUI/custom_nodes

!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git

# --------------------- ComfyUI_frontendの設定開始 ---------------------
%cd /content
!git clone https://github.com/Comfy-Org/ComfyUI_frontend.git
%cd /content/ComfyUI_frontend
!git reset --hard 1a817a48cbdb4dc9ca2cfbe77afb85d240358412
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://archive.creativaier.com/comfyui_materials/settings/env.txt  -d /content/ComfyUI_frontend -o .env
!sed -i 's/"build": "npm run typecheck && vite build"/"build": "vite build"/' /content/ComfyUI_frontend/package.json
# --------------------- ComfyUI_frontendの設定完了 ---------------------

# --------------------- デフォルトのworkflowファイルの設定開始 ---------------------
prefix = "0065"
uuid = "9a575720-1dfb-42ec-bed6-712d48376620"
root_path = f"https://archive.creativaier.com/comfyui_materials/{prefix}_{uuid}"

import requests
import json

# URLからJSONデータを取得
url = f"{root_path}/workflow.json"
response = requests.get(url)
data = response.json()

# # JavaScriptモジュール形式でデータを整形
js_content = f"""import type {{ ComfyWorkflowJSON }} from '@/types/comfyWorkflow'

export const defaultGraph: ComfyWorkflowJSON = {json.dumps(data, indent=2)};

export const defaultGraphJSON = JSON.stringify(defaultGraph)

export const blankGraph: ComfyWorkflowJSON = {{
  last_node_id: 0,
  last_link_id: 0,
  nodes: [],
  links: [],
  groups: [],
  config: {{}},
  extra: {{}},
  version: 0.4
}}"""

# ファイルに書き込む
output_path = "/content/ComfyUI_frontend/src/scripts/defaultGraph.ts"
with open(output_path, "w") as f:
    f.write(js_content)

# デプロイ
!npm install
!npm run deploy

# 画像ファイルを設定
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{root_path}/photo.png" -d /content/ComfyUI/input -o photo.png
# --------------------- デフォルトのworkflowファイルの設定完了 ---------------------

%cd /content/ComfyUI

# モデルファイルをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/diffusion_models/wan2.1_i2v_720p_14B_bf16.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.1_i2v_720p_14B_bf16.safetensors

# clip_visionをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/clip_vision/clip_vision_h.safetensors \
    -d /content/ComfyUI/models/clip_vision -o clip_vision_h.safetensors

# text_encordersをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors \
    -d /content/ComfyUI/models/text_encoders -o umt5_xxl_fp8_e4m3fn_scaled.safetensors

# VAEをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors \
    -d /content/ComfyUI/models/vae -o wan_2.1_vae.safetensors


!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')

%cd /content/ComfyUI

# スレッドの開始
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# ComfyUIの実行
!python main.py --dont-print-server

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) 